In [ ]:
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community \
  langsmith \
  

zsh:1: no matches found: langchain[google-genai]


In [4]:
!pip install -qU "langchain[google-genai]"




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import os
from getpass import getpass
from dotenv import load_dotenv


# must enter API key

load_dotenv()
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
if not langchain_api_key:
    raise ValueError("LANGCHAIN_API_KEY is not set")

os.environ["LANGCHAIN_API_KEY"] = langchain_api_key



# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

#project name as preffered
os.environ["LANGCHAIN_PROJECT"] = "First agent"

In [9]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return (x + y)

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return (y - x)

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return (x * y)

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return (x ** y)

In [10]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x10cda5d00>)

In [11]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [12]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [14]:
# CREATING AN AGENT USING LANGCHAIN'S LCEL

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [16]:
from langchain.chat_models import init_chat_model

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set")

os.environ["GEMINI_API_KEY"] = api_key
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [18]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

agent: RunnableSerializable = (
    {
        "input" : lambda x: x["input"],
        "chat_history" : lambda x: x["chat_history"],
        "agent_scratchpad" : lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [19]:
tool_call = agent.invoke({"input" : "What is 4 * 5", "chat_history" : []})
tool_call

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 5.0, "x": 4.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ff384542-9fa7-47f1-8cdc-11740cb3234c-0', tool_calls=[{'name': 'multiply', 'args': {'y': 5.0, 'x': 4.0}, 'id': '068c2389-94a4-4662-97f2-75c0152aa7c5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens': 66, 'total_tokens': 340, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 48}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
